In [53]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import view_points, BoxVisibility
from pyquaternion import Quaternion
import numpy as np
import pandas as pd
import tqdm

In [16]:
import os

In [18]:
os.chdir("../")

In [83]:
# Loading the nuscene from local system
nusc = NuScenes(version='v1.0-mini', dataroot='data/input_data/v1.0-mini/', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 1.895 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [4]:
my_scene = nusc.scene[0]
first_sample_token = my_scene['first_sample_token']
my_sample = nusc.get('sample', first_sample_token)

In [5]:
sensor = 'CAM_FRONT'
cam_front_data_token = my_sample['data'][sensor]
cam_front_data = nusc.get('sample_data', cam_front_data_token)
cam_front_data

{'token': 'e3d495d4ac534d54b321f50006683844',
 'sample_token': 'ca9a282c9e77460f8360f564131a8af5',
 'ego_pose_token': 'e3d495d4ac534d54b321f50006683844',
 'calibrated_sensor_token': '1d31c729b073425e8e0202c5c6e66ee1',
 'timestamp': 1532402927612460,
 'fileformat': 'jpg',
 'is_key_frame': True,
 'height': 900,
 'width': 1600,
 'filename': 'samples/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402927612460.jpg',
 'prev': '',
 'next': '68e8e98cf7b0487baa139df808641db7',
 'sensor_modality': 'camera',
 'channel': 'CAM_FRONT'}

In [212]:
channel_names = ["CAM_FRONT", "CAM_FRONT_LEFT", "CAM_FRONT_RIGHT",
                "CAM_BACK", "CAM_BACK_LEFT", "CAM_BACK_RIGHT"]

image_data_dict = dict()
# Iterate to scene

for scene in tqdm.tqdm(nusc.scene):
    #Iterate to sample
    token = scene['first_sample_token']
    while token != '':
        sample = nusc.get('sample', token)
        annotation_list = sample['anns']
        
        for channel in channel_names:
            channel_token = sample['data'][channel]
            filename, boxes, camera_intrinsic = nusc.get_sample_data(channel_token, selected_anntokens=annotation_list)
            
            if channel_token not in image_data_dict:
                image_data_dict[channel_token] = {'channel':channel,
                                                 'iID': camera_token,
                                                         'filePath': filename,
                                                         'annotations': []}
            for box in boxes:
                corners = view_points(box.corners(), np.array(camera_intrinsic), normalize=True)[:2, :]
                image_data_dict[channel_token]['annotations'].append({'name': box.name, 'corners': corners.tolist()})

        token = sample['next']
#         print(token)
#         break
        

100%|██████████| 10/10 [00:10<00:00,  1.07s/it]


## generate class distribution

In [214]:
channel_list, iID_list, name_list, corners_list = [], [], [], []
data = 
for key, value in image_data_dict.items():
    iID = value["iID"]
    annotations = value["annotations"]
    channel = value["channel"]
    
    for annotation in annotations:
        name = annotation["name"]
        corners = annotation["corners"]
        
        # Append the data to the respective lists
        iID_list.append(iID)
        name_list.append(name)
        channel_list.append(channel)
        corners_list.append(corners)

# Create a Pandas DataFrame from the lists
df = pd.DataFrame({
    "channel": channel_list,
    "iID": iID_list,
    "name": name_list,
    "corners": corners_list
})

In [204]:
df[['class_name', 'sub_category', 'grain']] = df['name'].str.split('.', expand=True, n=2)

In [205]:
class_counts = pd.DataFrame({'counts': df.class_name.value_counts(),
                                 'proportion': df.class_name.value_counts(normalize=True)
                                }).reset_index().rename(columns={"index":"class_name"})
class_counts = class_counts[['class_name', 'counts', 'proportion']]
class_counts

,class_name,counts,proportion
0,vehicle,6193,0.521648
1,human,3039,0.255980
2,movable_object,2593,0.218413
3,static_object,47,0.003959


In [206]:
sub_category_counts = pd.DataFrame({'counts': df[['class_name', 'sub_category']].value_counts(),
                                 'proportion': df[['class_name', 'sub_category']].value_counts(normalize=True)
                                }).reset_index()
sub_category_counts = sub_category_counts[['class_name', 'sub_category', 'counts', 'proportion']]
sub_category_counts

,class_name,sub_category,counts,proportion
0,vehicle,car,4785,0.403049
1,human,pedestrian,3039,0.255980
2,movable_object,barrier,1614,0.135950
3,movable_object,trafficcone,933,0.078588
4,vehicle,truck,432,0.036388
5,vehicle,motorcycle,311,0.026196
6,vehicle,bus,292,0.024596
7,vehicle,construction,167,0.014067
8,vehicle,bicycle,148,0.012466
9,vehicle,trailer,58,0.004885


In [160]:
image_data = pd.read_csv('data/output_data/image_data.csv')
image_data.shape

(22072, 7)

In [161]:
image_data['channel'].value_counts()

CAM_BACK           6508
CAM_FRONT          5364
CAM_FRONT_RIGHT    4049
CAM_BACK_RIGHT     3231
CAM_FRONT_LEFT     1921
CAM_BACK_LEFT       999
Name: channel, dtype: int64

In [164]:
sub_cat_counts = pd.read_csv('data/output_data/sub_category_counts.csv')
sub_cat_counts

,class_name,sub_category,counts,proportion
0,vehicle,car,9427,0.427102
1,human,pedestrian,5701,0.258291
2,movable_object,barrier,2692,0.121964
3,movable_object,trafficcone,1574,0.071312
4,vehicle,truck,851,0.038556
5,vehicle,motorcycle,543,0.024601
6,vehicle,bus,526,0.023831
7,vehicle,bicycle,274,0.012414
8,vehicle,construction,245,0.011100
9,movable_object,pushable_pullable,91,0.004123


In [163]:
class_counts = pd.read_csv('data/output_data/class_counts.csv')
class_counts

,class_name,counts,proportion
0,vehicle,11929,0.540458
1,human,5701,0.258291
2,movable_object,4373,0.198124
3,static_object,69,0.003126
